# LNG320 Gen Z Slang Similarity Analysis


In [20]:
%pip install -qU datasets pinecone-client umap-learn hdbscan scikit-learn plotly tqdm python-dotenv kaleido

/Users/beam/Workspace/Project/lng320-genz-slag-embedded/.venv/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import uuid
from getpass import getpass

import numpy as np
from datasets import load_dataset
from dotenv import load_dotenv
from IPython.display import display
from pinecone import Pinecone
from sklearn.decomposition import PCA
from tqdm import tqdm

import hdbscan
import plotly.express as px

try:
    import umap  # type: ignore
except ImportError:
    import umap.umap_ as umap

load_dotenv()


True

In [2]:
os.environ["PINECONE_API_KEY"] = os.getenv("PINECONE_API_KEY") or getpass(
    "Enter your Pinecone API key: "
)

pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])

## Load and Prepare Dataset


In [3]:
ds = load_dataset("MLBtrio/genz-slang-dataset", split="train")
df = ds.to_pandas()

df.columns = [c.lower().strip() for c in df.columns]
slang_col = "slang"

df["input_for_embedding"] = (
    df[slang_col].astype(str) + " is a slang term that means " + df["description"]
)

display(df.head())
print(f"Dataset shape: {df.shape}")


,slang,description,example,context,input_for_embedding
0,W,Shorthand for win,"Got the job today, big W!",Typically used in conversations to celebrate s...,W is a slang term that means Shorthand for win
1,L,Shorthand for loss/losing,"I forgot my wallet at home, that’s an L.",Often used when referring to a failure or mish...,L is a slang term that means Shorthand for los...
2,L+ratio,Response to a comment or action on the interne...,Your tweet got 5 likes and 100 replies calling...,Popularized on social media platforms to signi...,L+ratio is a slang term that means Response to...
3,Dank,excellent or of very high quality,That meme is so dank!,Commonly used in internet slang to refer to me...,Dank is a slang term that means excellent or o...
4,Cheugy,Derogatory term for Millennials. Used when mil...,"That phrase is so cheugy, no one says that any...",Used to refer to things that were once popular...,Cheugy is a slang term that means Derogatory t...


Dataset shape: (1779, 5)


In [4]:
df["id"] = [str(uuid.uuid5(uuid.NAMESPACE_DNS, str(s))) for s in df["slang"]]
print("Sample IDs:")
display(df[["slang", "id"]].head())


Sample IDs:


,slang,id
0,W,de7bed5e-22b7-516e-abdd-b6702d14f2c2
1,L,d0f78763-dafa-50e6-98cf-80e37b9ff0ac
2,L+ratio,acc7c7d5-9aaa-5d7d-9176-c8963f4ed424
3,Dank,99ca01c1-e02d-5c30-93f2-843624ab74bd
4,Cheugy,ea5729af-abec-59be-9fa5-2fb1403d92a1


## Connect to Pinecone Index


In [5]:
index_name = "lng320-genz-slang-qwen3-8b"
index = pc.Index(index_name)
print(f"Connected to index '{index_name}'")


Connected to index 'lng320-genz-slang-qwen3-8b'


## Retrieve Embeddings from Pinecone


In [6]:
fetched_vectors: dict[str, list[float]] = {}
batch_size = 100

for start in tqdm(range(0, len(df), batch_size), desc="Fetching embeddings"):
    batch_ids = df["id"].iloc[start : start + batch_size].tolist()
    response = index.fetch(ids=batch_ids)
    fetched_vectors.update(
        {vid: vec["values"] for vid, vec in response.vectors.items()}
    )

df["values"] = [fetched_vectors.get(row.id) for row in df.itertuples(index=False)]
missing = df[df["values"].isna()]["slang"].tolist()
if missing:
    raise RuntimeError(f"Missing vectors for {len(missing)} terms: {missing[:5]}...")

print(f"Fetched {len(fetched_vectors)} embeddings")


Fetching embeddings: 100%|██████████| 18/18 [03:07<00:00, 10.43s/it]

Fetched 1605 embeddings


In [7]:
embedding_matrix = np.vstack(df["values"].to_numpy())
print(f"Embedding matrix shape: {embedding_matrix.shape}")

Embedding matrix shape: (1779, 4096)


## PCA 3D Visualization

Visualize the high-dimensional embeddings in 3D using Principal Component Analysis.


In [ ]:
pca_3d = PCA(n_components=3, random_state=42)
pca_coords = pca_3d.fit_transform(embedding_matrix)

pca_df = df[["slang", "description"]].copy()
pca_df["PC1"] = pca_coords[:, 0]
pca_df["PC2"] = pca_coords[:, 1]
pca_df["PC3"] = pca_coords[:, 2]

explained_var = pca_3d.explained_variance_ratio_
print(
    f"Explained variance ratio: PC1={explained_var[0]:.3f}, PC2={explained_var[1]:.3f}, PC3={explained_var[2]:.3f}"
)
print(f"Total explained variance: {sum(explained_var):.3f}")


Explained variance ratio: PC1=0.025, PC2=0.024, PC3=0.022
Total explained variance: 0.072


In [27]:
fig_pca = px.scatter_3d(
    pca_df,
    x="PC1",
    y="PC2",
    z="PC3",
    hover_data=["slang", "description"],
    title="PCA 3D Projection of Gen Z Slang Embeddings",
    labels={
        "PC1": f"PC1 ({explained_var[0]:.1%})",
        "PC2": f"PC2 ({explained_var[1]:.1%})",
        "PC3": f"PC3 ({explained_var[2]:.1%})",
    },
)

fig_pca.update_traces(marker=dict(size=4, opacity=0.7))
fig_pca.update_layout(
    width=1000,
    height=800,
    font=dict(family="Times New Roman", size=12),
    title=dict(font=dict(size=16)),
    scene=dict(
        xaxis=dict(title=dict(font=dict(size=12))),
        yaxis=dict(title=dict(font=dict(size=12))),
        zaxis=dict(title=dict(font=dict(size=12))),
    ),
)

fig_pca.show()


## Dimensionality Reduction & Clustering

Using optimized parameters based on BERTopic best practices for text embeddings:

- **UMAP**: n_neighbors=15 (local structure), min_dist=0.0 (tight clusters), n_components=5 for clustering
- **HDBSCAN**: min_cluster_size=15, min_samples=10 for balanced cluster detection


In [8]:
umap_cluster = umap.UMAP(
    n_neighbors=15,
    n_components=5,
    min_dist=0.0,
    metric="cosine",
    random_state=42,
)
umap_embeddings = umap_cluster.fit_transform(embedding_matrix)
print(f"UMAP clustering embeddings shape: {umap_embeddings.shape}")

/Users/beam/Workspace/Project/lng320-genz-slag-embedded/.venv/lib/python3.12/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


UMAP clustering embeddings shape: (1779, 5)


### HDBSCAN Clustering


In [9]:
clusterer = hdbscan.HDBSCAN(
    min_cluster_size=15,
    min_samples=10,
    cluster_selection_method="eom",
    metric="euclidean",
    cluster_selection_epsilon=0.0,
    prediction_data=True,
)
cluster_labels = clusterer.fit_predict(umap_embeddings)

n_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)
n_noise = (cluster_labels == -1).sum()

print(f"Number of clusters: {n_clusters}")
print(f"Noise points: {n_noise} ({n_noise / len(cluster_labels) * 100:.1f}%)")

Number of clusters: 30
Noise points: 368 (20.7%)


/Users/beam/Workspace/Project/lng320-genz-slag-embedded/.venv/lib/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/beam/Workspace/Project/lng320-genz-slag-embedded/.venv/lib/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


### 2D Projection for Visualization


In [10]:
umap_viz = umap.UMAP(
    n_neighbors=15,
    n_components=2,
    min_dist=0.1,
    metric="cosine",
    random_state=42,
)
umap_2d = umap_viz.fit_transform(embedding_matrix)

viz_df = df[["id", "slang", "description"]].copy()
viz_df[["umap_x", "umap_y"]] = umap_2d
viz_df["cluster"] = cluster_labels
viz_df["cluster_prob"] = clusterer.probabilities_

print(f"Visualization embeddings shape: {umap_2d.shape}")
viz_df.head()


/Users/beam/Workspace/Project/lng320-genz-slag-embedded/.venv/lib/python3.12/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


Visualization embeddings shape: (1779, 2)


,id,slang,description,umap_x,umap_y,cluster,cluster_prob
0,de7bed5e-22b7-516e-abdd-b6702d14f2c2,W,Shorthand for win,-1.586983,6.195272,29,1.000000
1,d0f78763-dafa-50e6-98cf-80e37b9ff0ac,L,Shorthand for loss/losing,5.152213,6.530412,7,0.534892
2,acc7c7d5-9aaa-5d7d-9176-c8963f4ed424,L+ratio,Response to a comment or action on the interne...,5.337737,6.624959,7,0.494982
3,99ca01c1-e02d-5c30-93f2-843624ab74bd,Dank,excellent or of very high quality,2.778757,6.843239,26,0.970819
4,ea5729af-abec-59be-9fa5-2fb1403d92a1,Cheugy,Derogatory term for Millennials. Used when mil...,2.721590,7.790856,26,1.000000


In [23]:
fig_umap = px.scatter(
    viz_df,
    x="umap_x",
    y="umap_y",
    hover_data=["slang", "description"],
    title="UMAP 2D Projection of Gen Z Slang Embeddings",
    labels={"umap_x": "UMAP 1", "umap_y": "UMAP 2"},
)

fig_umap.update_traces(marker=dict(size=6, opacity=0.6, color="#636EFA"))
fig_umap.update_layout(
    width=1000,
    height=700,
    font=dict(family="Times New Roman", size=12),
    title=dict(font=dict(size=16)),
    xaxis=dict(title=dict(font=dict(size=14))),
    yaxis=dict(title=dict(font=dict(size=14))),
)

fig_umap.show()


### Cluster Analysis


In [11]:
cluster_counts = viz_df["cluster"].value_counts().sort_index()
print("Cluster distribution:")
display(cluster_counts)

print("\nExemplar slang terms per cluster:")
for cluster_id in sorted(viz_df["cluster"].unique()):
    cluster_terms = viz_df[viz_df["cluster"] == cluster_id]
    top_terms = cluster_terms.nlargest(5, "cluster_prob")[
        ["slang", "description", "cluster_prob"]
    ]
    label = "Noise" if cluster_id == -1 else f"Cluster {cluster_id}"
    print(f"\n{label} ({len(cluster_terms)} terms):")
    display(top_terms)


Cluster distribution:


cluster
-1     368
 0      24
 1      18
 2      36
 3      16
 4      81
 5      79
 6      32
 7      91
 8      22
 9      25
 10     78
 11     21
 12     19
 13     41
 14     37
 15     24
 16     44
 17    107
 18     29
 19     29
 20     69
 21     78
 22     36
 23     83
 24     26
 25     90
 26     83
 27     29
 28     16
 29     48
Name: count, dtype: int64


Exemplar slang terms per cluster:

Noise (368 terms):


,slang,description,cluster_prob
6,Woke,being politically aware,0.0
12,Stan,Supporting something. Specifically used in som...,0.0
17,Iykyk,"Acronym for ""If you know, you know."" Used to d...",0.0
18,Rent free,To be constantly thinking or upset about somet...,0.0
40,I’m weak,"Similar to ""I'm dead"", this is just another te...",0.0



Cluster 0 (24 terms):


...

,slang,description,cluster_prob
203,U,You,1.0
204,U2,You Too,1.0
1572,^URS,Up yours,1.0
1574,UDI,Unidentified drinking injury,1.0
1575,UDM,U (You) da (the) man,1.0



Cluster 28 (16 terms):


,slang,description,cluster_prob
209,WUF,Where Are You From?,1.0
1467,SUP,What’s up?,1.0
1642,WAYF,Where are you from?,1.0
1646,WBU,What about you?,1.0
1678,WRU,Where are you?,1.0



Cluster 29 (48 terms):


,slang,description,cluster_prob
0,W,Shorthand for win,1.0
292,*w*,wink,1.0
1628,W/,With,1.0
1647,WC,Welcome,1.0
1648,WC,Who cares,1.0


### Cluster Visualization


In [15]:
viz_df["cluster_label"] = viz_df["cluster"].apply(
    lambda x: "Noise" if x == -1 else f"Cluster {x}"
)

cluster_order = ["Noise"] + [f"Cluster {i}" for i in range(n_clusters)]
viz_df["cluster_label"] = viz_df["cluster_label"].astype("category")
viz_df["cluster_label"] = viz_df["cluster_label"].cat.set_categories(
    [c for c in cluster_order if c in viz_df["cluster_label"].values], ordered=True
)
viz_df_sorted = viz_df.sort_values("cluster_label")

fig = px.scatter(
    viz_df_sorted,
    x="umap_x",
    y="umap_y",
    color="cluster_label",
    hover_data=["slang", "description", "cluster_prob"],
    title="UMAP + HDBSCAN Clustering of Gen Z Slang",
    labels={"umap_x": "UMAP 1", "umap_y": "UMAP 2", "cluster_label": "Cluster"},
    category_orders={
        "cluster_label": [
            c for c in cluster_order if c in viz_df["cluster_label"].values
        ]
    },
)

fig.update_traces(marker=dict(size=6, opacity=0.8))
fig.update_layout(
    width=1200,
    height=800,
    font=dict(family="Times New Roman", size=12),
    title=dict(font=dict(size=16)),
    legend=dict(
        title=dict(text="Cluster", font=dict(size=14)),
        orientation="h",
        yanchor="top",
        y=-0.15,
        xanchor="center",
        x=0.5,
        font=dict(size=10),
        itemwidth=30,
    ),
    margin=dict(b=150),
    xaxis=dict(title=dict(font=dict(size=14))),
    yaxis=dict(title=dict(font=dict(size=14))),
)

fig.show()

In [13]:
viz_df.to_csv("viz_df.csv", index=False)
print("Exported viz_df.csv")

Exported viz_df.csv
